**Install requirements/library**

In [2]:
import os
import requests
import pandas as pd
import numpy as np

**Read the Excel file containing and make a list of companies using their scrip code as key**

In [22]:
df = pd.read_excel('nifty_companies.xlsx')
companies = df['Scrip_code'].tolist()


In [31]:
for company in range(0,len(companies)): #give other name to iterator
    if(np.isnan(companies[company])):
       continue
    scrip_code=int(companies[company])
    url=f"https://api.bseindia.com/BseIndiaAPI/api/AnnualReport_New/w?scripcode={scrip_code}"
    headers = {                    
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
              } #keep as it is based on bse specs
    response = requests.get(url,headers=headers)
    if response.status_code == 200: #200 is a good status
      data = response.json()["Table"]
      # Create a new folder for the downloaded file
      if(len(data)==0):
         continue
      scrip_name=data[0]["scrip_name"]
      folder_name = scrip_name
      print("------- Initiating Annual Report download for ",scrip_name,"-----")
      if not os.path.exists(folder_name):
          os.mkdir(folder_name)
      # loop over each year report
      for report in data: #report is an iterator
          file_url = report["PDFDownload"]
          Year=report["Year"]
          if(int(Year)<2010):
            break
          file_name=str(scrip_code)+"_"+Year+".pdf"
          # Get response object for file
          file_response=None
          try:
            file_response = requests.get(file_url, headers=headers, timeout=15)
            file_response.raise_for_status()
          except requests.exceptions.Timeout:
            print(f"Request timed out for file {file_name}. Moving on to the next file.")
            continue
          except requests.exceptions.HTTPError as e:
            print(f"HTTP error ({e.response.status_code}) for file {file_name}. Skipping.")
            continue
          except Exception as e:
            print(f"Error for file {file_name}: {str(e)}")
            continue
          if file_response.status_code == 200:
              # Set the file path for the downloaded file
              path = os.path.join(folder_name, file_name)
              # Check if file with the same name already exists
              if os.path.exists(file_name):
                    i = 1
                    while True:
                        new_filename = f"{Year}({i}).pdf"
                        if not os.path.exists(new_filename):
                            file_name = new_filename
                            break
                        i += 1
              file_path = os.path.join(folder_name, file_name)
              print('Downloading Annual report for',Year)
              with open(file_path, "wb") as f:
                  f.write(file_response.content)
              print('PDF file downloaded successfully.')
          else:
              print('Failed to download Annual report for',Year,' ',file_response.status_code)        
    else:
        print(f"Error fetching annual reports: {response.status_code}")

------- Initiating Annual Report download for  NOCIL LTD -----
PDF file downloaded successfully.
PDF file downloaded successfully.
PDF file downloaded successfully.
PDF file downloaded successfully.
PDF file downloaded successfully.
PDF file downloaded successfully.
PDF file downloaded successfully.
PDF file downloaded successfully.
PDF file downloaded successfully.
PDF file downloaded successfully.
PDF file downloaded successfully.
HTTP error (500) for file 500730_2011.pdf. Skipping.
PDF file downloaded successfully.
------- Initiating Annual Report download for  NTPC LTD -----
PDF file downloaded successfully.
PDF file downloaded successfully.
PDF file downloaded successfully.
PDF file downloaded successfully.
PDF file downloaded successfully.
PDF file downloaded successfully.
PDF file downloaded successfully.
PDF file downloaded successfully.
PDF file downloaded successfully.
PDF file downloaded successfully.
PDF file downloaded successfully.
PDF file downloaded successfully.
PDF fi